#### **Var definitions**
Change it whenever necessary

In [28]:
base_path = "/Users/lorenapacheco/Concordia/Masters/"
project = "junit4"
coverage_path = base_path + "coverage_data/"+ project + "/"
project_path = base_path + "open_source_repos_being_studied/"+  project + "/"
output_folder = base_path + "BugReportsMining/coverage_info/" +  project + "/"

#### **Script**

In [29]:
import os
import glob
import json
import csv

def json_file_to_dict(file):
    data = {}
    with open(os.path.join(output_folder, file+'.json'), 'r') as fp:
        data = json.load(fp)
    fp.close()
    return data

def find_file(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)
        
def read_file_lines(file_name, path=project_path):
    path = find_file(file_name, project_path)
    with open(path, 'r', encoding='utf-8', errors='ignore') as file:
        lines =file.readlines()
    file.close()
    return lines
    

def count_lines_of_code (file_name):
    lines = read_file_lines(file_name)
    multine_comment_active = False
    count = 0
    for i in range(0, len(lines)):
        line = lines[i]
        # Ignore blocks inside multiline comments
        if line.strip().startswith("/*"):
            multine_comment_active = True
            continue
        if multine_comment_active:
            if line.strip().endswith("*/"):
                multine_comment_active = False
                continue
        # Ignore single line comments
        if line.strip().startswith("//"):
            continue
        # Count non-empty lines
        if line.strip() != "":
            # Ignore package and import lines
            if line.strip().startswith("package") or line.strip().startswith("import"):
                continue
            # Treating statements with line break
            if line.strip().endswith("{") or line.strip().endswith("}") or line.strip().endswith(";") or line.strip().endswith(";") or line.strip().endswith(","):
                count = count + 1

    return count

def create_coverage_percent_file(file_name, obj):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    with open(os.path.join(output_folder, file_name+'.csv'), 'w') as file:
        # create the csv writer object
        csv_writer = csv.writer(file)
        csv_writer.writerow(["File name", "Coverage percent"])
        for file_name in obj.keys():
            total_num_lines = count_lines_of_code(file_name)
            coverage_percent = len(coverage[file_name])/total_num_lines * 100
            csv_writer.writerow([file_name, coverage_percent])
    file.close()
        
for commit_folder in glob.glob(coverage_path +'*'):
    print(commit_hash + " - OK")
    commit_hash = os.path.basename(commit_folder)
    os.chdir(project_path)
    checkout_command = "git checkout  --quiet " + commit_hash
    os.system(checkout_command)

    data = {}
    if os.path.exists(coverage_path + commit_hash +'/coverage.json'):
        with open(os.path.join(coverage_path + commit_hash +'/coverage.json'), 'r') as file:
            data = json.load(file)
        file.close()
    else:
        for test_file in glob.glob(coverage_path + commit_hash + '/details/*.json'):
            test_name = os.path.basename(test_file).replace('.json', '')
            data [test_name] = json_file_to_dict(test_file.replace('.json', ''))
    coverage = {}
    for test in data.keys():
        for file in data[test]["covered_lines"].keys():
            file_name = file.split('.')[-1] + ".java"
            if file_name not in coverage.keys():
                coverage[file_name] = []
            for line in (data[test]["covered_lines"][file]):
                coverage[file_name].append(int(line))
    for file in coverage.keys():
        coverage[file] = sorted(set(coverage[file]))
    
    create_coverage_percent_file(commit_hash, coverage)
    
print("Extraction completed")
    

fb50d9635874185fc050735057b8a7363cd53f3f - OK
a25189631a9eb551a1ac961ca4d2c2db37a687a0 - OK
Extraction completed
